The purpose of this code is to count inclusion number and size for various cell samples.

Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from skimage import io, filters, morphology, segmentation, exposure
from skimage.filters import gaussian, threshold_otsu, try_all_threshold
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_dilation, disk
import imageio
from czifile import CziFile
import czifile
from cellpose import models
from IPython.display import clear_output

Define Sub Functions

In [ ]:
def iteration_cycle(image_analysis, basename):
    individual_inclusions_per_cell = []
    cell_inclusion_details = []  # To store cell number and inclusion count
    model = models.Cellpose(gpu=False, model_type='cyto')  # Set gpu=True if you want to use GPU

    image = czifile.imread(image_analysis)
    image_squeezed = np.squeeze(image)
    first_frame = image_squeezed[1, :, :]
    target_shape = first_frame.shape
    
    blurred_frame = gaussian(first_frame, sigma=1)
    normalized_image = (blurred_frame - blurred_frame.min()) / (blurred_frame.max() - blurred_frame.min())
    # Replace NaN values with zero
    normalized_image = np.nan_to_num(normalized_image)
    plt.figure(figsize=(6, 6))  # Adjust figsize as needed
    plt.imshow(first_frame, cmap='viridis')
    plt.axis('off')  # Turn off axes

    # Use Cellpose for cell segmentation
    masks, flows, styles, diams = model.eval(normalized_image, diameter=None, channels=[0, 0])

    labeled_image = label(masks)
    props = regionprops(labeled_image)
    for i, prop in enumerate(props):
        if prop.area > 1000:
            # Store the area of the cell
            mask = labeled_image == prop.label  # Corrected to use prop.label
            mask = zoom(mask, zoom=np.array(target_shape) / np.array(mask.shape), order=0)
            mask_applied = mask * first_frame

            mask_applied = gaussian(mask_applied)
            
            threshold_mean = (mask_applied - mask_applied.min()) / (mask_applied.max() - mask_applied.min())
            threshold = threshold_otsu(threshold_mean[mask > 0]) + 0.15
            inclusion_mask = mask_applied > threshold

            labeled_inclusions = label(inclusion_mask)
            inclusion_size = []
            
            for region in regionprops(labeled_inclusions):  # Loop through each inclusion
                inclusion_size.append(region.area)
        
            for size in inclusion_size: 
                if size / prop.area > 0.7: # Catch cases of poor segmentation
                    continue
                individual_inclusions_per_cell.append({
                    'Filename': basename,
                    'Cell Number': i + 1,
                    'Inclusion Size': size
                })
            cell_inclusion_details.append({
                'Filename': basename,
                'Cell Number': i + 1,
                'Number of Inclusions': len(inclusion_size)
            })
                
    df_individual = pd.DataFrame(individual_inclusions_per_cell)
    df_cell_summary = pd.DataFrame(cell_inclusion_details)

    return df_individual, df_cell_summary

def sizes_of_all_inclusions(inclusions_size_list): 
    inclusions_df_array = []
    for df in inclusions_size_list:
        inclusions_df_array.extend(df.to_dict(orient='records'))
    return pd.DataFrame(inclusions_df_array)


Define Main Function

In [ ]:
# Define the directory path
image_dir = r"test_images"
output_dir = os.getcwd()

# List all files in the image directory
images_to_analyze = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

inclusions_size_list = []
df_cell_summary_list = []

# Iterate through the list of image files
for path in images_to_analyze:
    if path.endswith('.czi'):
        image_path = os.path.join(image_dir, path)
        basename = os.path.basename(path)[:-4]
        # Run the iteration cycle
        df_individual, df_cell_summary = iteration_cycle(image_path, basename)
        # Append the DataFrames to the respective lists
        inclusions_size_list.append(df_individual)
        df_cell_summary_list.append(df_cell_summary)

# Combine all individual DataFrames
combined_inclusions_df = sizes_of_all_inclusions(inclusions_size_list)
# Save the combined DataFrame to an Excel file
output_excel_path = os.path.join(output_dir, 'INCLUSION_SIZE.xlsx')
combined_inclusions_df.to_excel(output_excel_path, index=False)

# combine and save the cell summary DataFrames
combined_cell_summary_df = pd.concat(df_cell_summary_list, ignore_index=True)
output_summary_path = os.path.join(output_dir, 'SUMMARY.xlsx')
combined_cell_summary_df.to_excel(output_summary_path, index=False)